In [ ]:
import pandas as pd
import numpy as np
import random 
import time

import re
import spacy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchsummary import summary
from torchtext import data 
from torchtext import datasets
from torchtext import vocab

#
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data/Sentiment Analysis Dataset.csv', error_bad_lines= False)

# df_rev = pd.read_csv('',error_bad_lines = False)

# df_rev_fake = pd.read_csv('',error_bad_lines = False)
vec = vocab.Vectors('/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Embeddings/glove.twitter.27B.25d.txt')

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'
100%|█████████▉| 1193245/1193514 [00:24<00:00, 48484.72it/s]

In [ ]:
NLP = spacy.load('en')
def tokenizer (comment):
  comment = re.sub(
      r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ",
      str(comment))
  comment = re.sub(r"[ ]+", " ", comment)
  comment = re.sub(r"\!+", "!", comment)
  comment = re.sub(r"\,+", ",", comment)
  comment = re.sub(r"\?+", "?", comment)
  return [x.text for x in NLP.tokenizer(comment) if x.text != " "]

## Preprocessing Data (Avoid running repeatedly as it will take awhile on such a big dataset). Check to see if the csv has already been processed

In [ ]:

# print('Calculating Tokenized Length')
# #df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data/Sentiment Analysis Dataset.csv', error_bad_lines= False) 
df['SentimentText_Len'] = df['SentimentText'].apply(tokenizer).apply(len)

# df_filt = df[df['SentimentText_Len'] <= 15]
# df_filt = df_filt[df_filt['SentimentText_Len'] >= 10]

print('Filtering Data and Renaming Columns')
df_filt = df[df['SentimentText_Len'] == 25]
df_filt.columns = ['ItemID', 'label', 'SentimentSource' ,'text', 'SentimentText_Len']



Filtering Data and Renaming Columns


In [ ]:
df.columns

Index(['ItemID', 'Sentiment', 'SentimentSource', 'SentimentText'], dtype='object')

In [ ]:
# #rerun to resample
# print('Resampling Data')
#df_filt_downsample = df_filt[['text','label']]#.sample(n = 10000)
#df_filt_downsample.to_csv('/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data/filtered_sentiment_dataset.csv')

df.columns = ['ItemID', 'label', 'SentimentSource' ,'text', 'SentimentText_Len']
# df[['text','label']].to_csv('/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data/all_sentiment_dataset.csv')
print(len(df))
print(len(df_filt))
print(df)
df_filt

1578612
41577
          ItemID  ...  SentimentText_Len
0              1  ...                  8
1              2  ...                  7
2              3  ...                  6
3              4  ...                 32
4              5  ...                  9
...          ...  ...                ...
1578607  1578623  ...                  5
1578608  1578624  ...                  5
1578609  1578625  ...                  8
1578610  1578626  ...                 10
1578611  1578627  ...                  4

[1578612 rows x 5 columns]


,ItemID,label,SentimentSource,text,SentimentText_Len
22,23,1,Sentiment140,You're the only one who can see this cause...,25
26,27,0,Sentiment140,can't be bothered. i wish i could spend the...,25
31,32,0,Sentiment140,i miss you guys too i think i'm wearing...,25
117,118,1,Sentiment140,"True, highly subjective of me there. Tombre ...",25
129,130,0,Sentiment140,- You: hi Stranger: HELLO I AM FRANK LAMPARD ...,25
...,...,...,...,...,...
1578509,1578525,0,Sentiment140,"yzabellopez: Â yayayay cant wait, i just wish ...",25
1578529,1578545,1,Sentiment140,"Zaki stood on a big nail today and is resting,...",25
1578576,1578592,1,Sentiment140,"zomg, CLE -&gt; SEA is pretty far it turns out...",25
1578587,1578603,0,Sentiment140,Zoran lost Croatian Idol! The difference was ...,25


In [ ]:
# df_filt_downsample.to_csv('/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data/filtered_sentiment_dataset.csv')


In [ ]:
df.dtypes

ItemID                int64
label                 int64
SentimentSource      object
text                 object
SentimentText_Len     int64
dtype: object

In [ ]:
### Load sampled data from csv.
TEXT = data.Field(tokenize = tokenizer, batch_first = True) #batch_first required so we dont have to permute the data for input to CNN.

LABEL = data.Field(use_vocab = False,sequential = False)
#redundant as our labels are strings...

dataset = data.TabularDataset(path = '/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data/filtered_sentiment_dataset.csv' 
                              , format = 'csv', fields = [('', None),("text",TEXT), ("label",LABEL)], skip_header = True)

train_data, test_data, valid_data = dataset.split(split_ratio = [0.6,0.2,0.2])

#load entire dataset to build vocab
train_data_all = data.TabularDataset(path = '/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data/all_sentiment_dataset.csv' 
                              , format = 'csv', fields = [('', None),("text",TEXT), ("label",LABEL)], skip_header = True)


In [ ]:
vec = vocab.Vectors('/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Embeddings/glove.twitter.27B.25d.txt')

# Select only the most important 30000 words
# MAX_VOCAB_SIZE = 300000

# Build vocabulary
TEXT.build_vocab(train_data_all, 
                #  max_size = MAX_VOCAB_SIZE, 
                 # Load pretrained embeddings
                #  vectors = "glove.6B.50d", 
                #  vectors = "glove.6B.100d", 
                #  vectors = "glove.6B.200d", 
                #  vectors = "glove.6B.300d", 
                  # vectors = vec,
                 #vectors = "glove.twitter.27B.25d",
                vectors = vec,
                unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [ ]:
print(len(TEXT.vocab))
print(len(LABEL.vocab))

845598
3


In [ ]:
LABEL.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'0': 1, '1': 2, '<unk>': 0})

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.text),
    sort_within_batch = False,
    device = device)

# for batch in train_iterator:
#   print(' ')
#   print(batch.label.shape)
#   print(batch.label)
#   print(batch.text.shape)
#   print(batch.text)

# for batch in valid_iterator:
#   print(' ')
#   print(batch.label.shape)
#   print(batch.label.type())
#   print(batch.text.shape)

# for batch in test_iterator:
#   print(' ')
#   print(batch.label.shape)
#   print(batch.label.type())
#   print(batch.text.shape)




cuda


In [ ]:

#model
class nlp_cnn(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)

        # print(embedded.shape)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)


In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    # print(' ')
    # print(rounded_preds)
    # print(y)
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        # print(predictions.type())
        # print(batch.label.squeeze(1).type())
        loss = criterion(predictions, batch.label.float())
        
        acc = binary_accuracy(predictions, batch.label.float())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            # print(predictions)
            # print(batch.label.float())
            loss = criterion(predictions, batch.label.float())
            
            acc = binary_accuracy(predictions, batch.label.float())

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
INPUT_DIM =  len(TEXT.vocab)
EMBEDDING_DIM = 25
N_FILTERS = 25
FILTER_SIZES = [3,4]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = nlp_cnn(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

print(model)
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

nlp_cnn(
  (embedding): Embedding(845598, 25, padding_idx=1)
  (convs): ModuleList(
    (0): Conv2d(1, 25, kernel_size=(3, 25), stride=(1, 1))
    (1): Conv2d(1, 25, kernel_size=(4, 25), stride=(1, 1))
  )
  (fc): Linear(in_features=50, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Model_Param/Model_1_State_Dict')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 0.672 | Train Acc: 58.30%
	 Val. Loss: 0.626 |  Val. Acc: 66.07%
Epoch: 02 | Epoch Time: 0m 6s
	Train Loss: 0.604 | Train Acc: 67.60%
	 Val. Loss: 0.577 |  Val. Acc: 70.16%
Epoch: 03 | Epoch Time: 0m 6s
	Train Loss: 0.549 | Train Acc: 72.62%
	 Val. Loss: 0.558 |  Val. Acc: 71.79%
Epoch: 04 | Epoch Time: 0m 6s
	Train Loss: 0.497 | Train Acc: 76.71%
	 Val. Loss: 0.558 |  Val. Acc: 72.22%
Epoch: 05 | Epoch Time: 0m 6s
	Train Loss: 0.432 | Train Acc: 80.70%
	 Val. Loss: 0.573 |  Val. Acc: 71.51%
Epoch: 06 | Epoch Time: 0m 6s
	Train Loss: 0.361 | Train Acc: 85.17%
	 Val. Loss: 0.611 |  Val. Acc: 70.58%
Epoch: 07 | Epoch Time: 0m 6s
	Train Loss: 0.281 | Train Acc: 89.20%
	 Val. Loss: 0.685 |  Val. Acc: 69.70%
Epoch: 08 | Epoch Time: 0m 6s
	Train Loss: 0.215 | Train Acc: 92.05%
	 Val. Loss: 0.766 |  Val. Acc: 68.53%
Epoch: 09 | Epoch Time: 0m 6s
	Train Loss: 0.166 | Train Acc: 94.19%
	 Val. Loss: 0.840 |  Val. Acc: 67.91%
Epoch: 10 | Epoch Time: 0m 6

## Saving Model Intermediates 

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Model_Param/Model_1_State_Dict'

path_data = '/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Model_Param_2/Model_1_Embedded_Inputs'
path_inter = '/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Model_Param_2/Model_1_Intermediates'
path_data_og = '/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Model_Param_2/Model_1_Word_Inputs'
path_labels = '/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Model_Param_2/Model_1_Labels'
path_valid =  '/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data_Val_2/Val'

In [ ]:
## Loading Parameters
# set path equal to where the file 'Model_1_State_Dict' is located
#path = ''
model_2 = nlp_cnn(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

#load model
model_2.load_state_dict(torch.load(path))
#set to eval
model_2.eval()

# #store param
# params = []
# for name, param in model_2.named_parameters():
#   if param.requires_grad:
#     params.append((name, param.data.numpy()))
# # print(params)

# torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#activation = {} 
batch_number = 0
def get_activation(name):
    def hook(model, input, output):
      #shift to CPU to reduce cuda usage
        #activation[name] = output.detach().cpu().numpy()
        save_path = path_inter + '_batch_' + str(batch_number) + '_' + name 
        torch.save(output.detach().cpu(), save_path)
        print(batch_number)
        print(name)
        # print(output.detach().cpu().numpy().shape)
    return hook

print(model_2)

model_2 = model_2.to(device)
#register hooks
model_2.embedding.register_forward_hook(get_activation('input_embed'))
# model_2.convs[0].register_forward_hook(get_activation('conv_0'))
# model_2.convs[1].register_forward_hook(get_activation('conv_1'))
# model_2.convs[2].register_forward_hook(get_activation('conv_2'))
model_2.fc.register_forward_hook(get_activation('fc'))

print(model_2)

all_activations = []
model_2.eval()

with torch.no_grad():
    batch_number = 0
    for batch in train_iterator:

        # activation = {}
        torch.save(batch.text, path_data_og + '_batch_' + str(batch_number))

        predictions = model_2(batch.text).squeeze(1)

        # all_activations.append(activation)
        # torch.save(activation, path_inter)
        
        batch_number +=1
        loss = criterion(predictions, batch.label.float())

        torch.save(batch.label, path_labels + '_batch_' + str(batch_number))
            
        acc = binary_accuracy(predictions, batch.label.float())
        print(binary_accuracy)

# for batch in train_iterator:
#   # for i in batch.text: 
#   data.append(batch.text)
#   #reset dictionary
#   activation = {}
#   #run model on 1 data
#   model_2(batch.text)
#   #store dictionary in a list
#   all_activations.append(activation)

# x = torch.randn(1, 25)
# output = model(x)
# print(activation['fc2'])



nlp_cnn(
  (embedding): Embedding(845598, 25, padding_idx=1)
  (convs): ModuleList(
    (0): Conv2d(1, 25, kernel_size=(3, 25), stride=(1, 1))
    (1): Conv2d(1, 25, kernel_size=(4, 25), stride=(1, 1))
  )
  (fc): Linear(in_features=50, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
nlp_cnn(
  (embedding): Embedding(845598, 25, padding_idx=1)
  (convs): ModuleList(
    (0): Conv2d(1, 25, kernel_size=(3, 25), stride=(1, 1))
    (1): Conv2d(1, 25, kernel_size=(4, 25), stride=(1, 1))
  )
  (fc): Linear(in_features=50, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
0
input_embed
0
fc
<function binary_accuracy at 0x7f31fa337378>
1
input_embed
1
fc
<function binary_accuracy at 0x7f31fa337378>
2
input_embed
2
fc
<function binary_accuracy at 0x7f31fa337378>
3
input_embed
3
fc
<function binary_accuracy at 0x7f31fa337378>
4
input_embed
4
fc
<function binary_accuracy at 0x7f31fa337378>
5
input_embed
5
fc
<function binary_accuracy at 0x7f31fa337378

In [ ]:
batch_number = 0 
for batch in valid_iterator:
  torch.save(batch.text,path_valid + '_batch_' + str(batch_number))
  batch_number +=1

In [ ]:
#load the torch file to see if it works out.

path = '/content/drive/My Drive/Colab Notebooks/ML: DO /Data/Model_1_Intermediates_conv_0'
data_conv_0 = torch.load(path)

print(data_conv_0.shape)

torch.Size([64, 100, 859, 1])


In [ ]:
batch.text.shape

torch.Size([64, 1169])

In [ ]:
print(len(all_activations))
print(len(train_data))

print(all_activations[0])

## Testing Sensitivity of GloVE representations.

In [ ]:
#build lookup dictionary from TEXT.vocab 
embed_dict = {}
for j in TEXT.vocab.stoi:
  embed_dict[j] = TEXT.vocab.vectors[TEXT.vocab.stoi[j]]

#remove the null and pad possibilities
del embed_dict["<unk>"]
del embed_dict["<pad>"]

In [ ]:
def sentence_embed(sentence,embed_dict):
  sent_mat = []
  for i in sentence.split(sep = ' '): 
    sent_mat.append(embed_dict[i])
  
  return torch.stack(sent_mat)

def rev_word_embed(sent_matrices,embed_dict,metric = 'MSE'):
  """
  Calculates the closest word based on a metric
  Inputs:
    sent_matrices: 2-d matric representing a sentence of tensors.
    embed_dict: dictionary of word: sentence lookups. 
  """
  #iterate word in sentence
  sentence = ''
  for i in sent_matrices:
      closest_word = ('test',500)
      sentence_mse = []
      #find closest based on MSE
      for j in embed_dict:
        metric_val = np.mean(((embed_dict[j] - i)**2).numpy())
        if metric_val < closest_word[1]:

          closest_word = (j,metric_val)
      sentence_mse.append(closest_word[1])
  
      sentence += closest_word[0] + ' ' 
  print(sentence_mse)
  return sentence

def pt_permute(sentence, sent_mat, val):
  """
  permutes a given tensor matrix representing a sentence at a random point by a random amount.
  Inputs: 
    sentence: original sentence
    sent_mat: 2-d tensor matrix
    val: range of values
  """

  i = random.randrange(0,sent_mat.shape[0])
  j = random.randrange(0,sent_mat.shape[1])

  new_sent_mat = sent_mat
  new_sent_mat[i][j] += val
  return new_sent_mat

  print('We permuted the word:' + str(sentence.split(sep = ' ')[i]  + " by " + str(val)) )

In [ ]:
sentence = 'I like that the movie was about pie'
test_sentence = sentence_embed(sentence = sentence,embed_dict = embed_dict)
print(sentence)
print(test_sentence)
for i in range(0,20,1):
  print('Permute by ' + str(i))
  test_sentence = sentence_embed(sentence = sentence,embed_dict = embed_dict)

  test_sentence_permuted = pt_permute(sentence = sentence, sent_mat = test_sentence, val = i)

  final_sent = rev_word_embed(test_sentence_permuted, embed_dict = embed_dict)
  print(final_sent)

NameError: ignored

In [ ]:
#load MIP data
path = "/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Data/Y0"
data_mip = torch.load(path)

data_in = torch.load("/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Model_Param/Model_1_Intermediates_batch_0_input_embed")

print(type(data))
for i in range(0,len(data)):
  # print(i)
  print(rev_word_embed(sent_matrices = data_in[i], embed_dict = embed_dict, metric = 'MSE'))
  print(rev_word_embed(sent_matrices = data_mip[i] ,embed_dict = embed_dict ,metric = 'MSE'))


<class 'torch.Tensor'>
[0.0060009314]
I totally stalked gaga at rove tonight , but security yelled at us to go away they do nt do that for you lol bitches 
[0.008160587076293725]
I totally stalked gaga at rove tonight , but security yelled at us to go away they do nt do that for you lol bitches 
[0.001817887]
I did the only person who really cares about me leave to the army . I one loves me anymore . I hate it . 
[0.0037542696174383662]
I did the only person who really cares about me leave to the army . I one loves me anymore . I hate it . 
[0.0081332745]
I I ohhh I wish I could I have to go to school though which is unlucky because it 's my last week I shame 
[0.006286405709794657]
I I ohhh I wish I could I have to go to school though which is unlucky because it 's my last week I shame 
[0.0024811395]
I I lakers I I last night I m so I they can do it again on I I is the sun sooo ugly out 
[0.0020512947746822835]
I I lakers I I last night I m so I they can do it again on I I is the sun

In [ ]:
embed_dict = {}
for j in TEXT.vocab.stoi:
  embed_dict[j] = TEXT.vocab.vectors[TEXT.vocab.stoi[j]]

word_dict = {value : key for (key, value) in TEXT.vocab.stoi.items()}

def sentence_embed(sentence,embed_dict):
    sent_mat = []
    for i in sentence.split(sep = ' '): 
        sent_mat.append(embed_dict[i])

    return torch.stack(sent_mat)

def input_lookup(sent_matrix,embed_dict):
    """
    
    """
    sentence = ''
    for i in sent_matrix:
        sentence += embed_dict[int(i)] + ' '
    return sentence

def rev_word_embed(sent_matrices, word_embeds ,embed_dict, ign_unk = True, metric = 'L2'):
    """
    Calculates the closest word based on a metric
    Inputs:
    sent_matrices: 2-d matric representing a sentence of tensors.
    word_embeds: tensor matrix of all possible word embeddings
    embed_dict: dictionary of value: word lookups. 
    """
    #iterate word in sentence
    sentence = ''
    for i in sent_matrices:
        if metric == 'L1':
            if ign_unk:
                closest_word = np.argmin(torch.sum(( torch.abs(word_embeds - i.to("cpu") )), axis = 1).numpy()[2:]) + 2
            else:
                closest_word = np.argmin(torch.sum(( torch.abs(word_embeds - i.to("cpu") )), axis = 1).numpy())
        elif metric == 'L2':
            if ign_unk:
                closest_word = np.argmin(torch.sum(((word_embeds - i.to("cpu"))**2), axis = 1).numpy()[2:]) + 2
            else:
                closest_word = np.argmin(torch.sum(((word_embeds - i.to("cpu"))**2), axis = 1).numpy())
#         elif metric == 'COS':
#             if ign_unk:
#                 closest_word = np.argmin(torch.sum(((word_embeds - i.to("cpu"))**2), axis = 1).numpy()[2:]) + 2
#             else:
#                 closest_word = np.argmin(torch.sum(((word_embeds - i.to("cpu"))**2), axis = 1).numpy())
        else:
            raise NameError('Please pick L1 or L2 norm as metric')
            
        sentence += (embed_dict[closest_word] + ' ')
    return sentence

def pt_permute(sentence, sent_mat, val):
    """
    permutes a given tensor matrix representing a sentence at a random point by a random amount.
    Inputs: 
    sentence: original sentence
    sent_mat: 2-d tensor matrix
    val: range of values
    """

    i = random.randrange(0,sent_mat.shape[0])
    j = random.randrange(0,sent_mat.shape[1])

    new_sent_mat = sent_mat
    new_sent_mat[i][j] += val
    return new_sent_mat

    print('We permuted the word:' + str(sentence.split(sep = ' ')[i]  + " by " + str(val)) )

In [ ]:
#load MIP data
mip_path = "/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/adversarial_examples/0-100/output0"
# data_mip = torch.load(mip_path)


# in_path = r"C:\Users\wilso\Desktop\DL_DO_Project\Model_Param\Model_1_Word_Inputs_batch_0"
in_path = "/content/drive/My Drive/Colab Notebooks/ML: DO /DL_DO_Project_DNN/Model_Param"
data_in = torch.load(in_path)